In [ ]:
!pip install pip==23.3
!pip install fuzzy
!pip install metaphone
!pip install ai4bharat-transliteration
!pip install googletrans==3.1.0a0
import pandas as pd
from bs4 import BeautifulSoup
import fuzzy
from metaphone import doublemetaphone
import os
import json
from sentence_transformers import SentenceTransformer
from nltk.corpus import stopwords
from googletrans import Translator
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 38.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Preparing metadata (setup.py) ... done
  Created wheel for fuzzy: filename=Fuzzy-1.2.2-cp310-cp310-linux_x86_64.whl size=206409 sha256=1ddecac2ca1f095d06327eeb5366feaa07f1416f5341032ca7c4c32e2ad1b0a6
  Stored in directory: /root/.cache/pip/wheels/3e/a7/03/cb9f418321ad60736caa7a86f0077f6ab74b669885c3e03cc1
Successfully built fuzzy
  Preparing metadata (setup.py) ... done
  Created wheel for metaphone: filename=Metaphone-0.6-py3-none-any.whl size=13901 sha256=725e95345bc17f8dd580ef461ee487fd83e85fce074862a7b8c747ad1e8ad325
  Stored in directory: /root/.cache/pip/wheels/23/dd/1d/6cdd346605db62bde1f60954155e9ce48f4681c243f265b704
Successfully built metaphone
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 76.3 MB/s eta 0:00:00
  Installing build dependencies ... 

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
from ai4bharat.transliteration import XlitEngine
e = XlitEngine("hi", beam_width=10, rescore=True)

Initializing Multilingual model for transliteration


/usr/local/lib/python3.10/dist-packages/fairseq/checkpoint_utils.py:315: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(f, map_location=torch.device("cpu")

In [ ]:
import fuzzy
from metaphone import doublemetaphone

In [ ]:
def soundex(name):
    codes = {
        "BFPV": "1", "CGJKQSXZ": "2", "DT": "3",
        "L": "4", "MN": "5", "R": "6"
    }

    def get_code(letter):
        for key in codes:
            if letter in key:
                return codes[key]
        return ""

    def encode_word(word):
        first_letter = word[0].upper()
        encoded = first_letter + ''.join(get_code(char.upper()) for char in word[1:])
        result = [encoded[0]]
        for i in range(1, len(encoded)):
            if encoded[i] != encoded[i - 1]:
                result.append(encoded[i])
        result = ''.join(result).replace('0', '')
        return (result[:4] + "0000")[:4]

    # Process each word separately
    words = name.split()
    soundex_codes = [encode_word(word) for word in words]
    return soundex_codes

# Example usage
name="Aultom"
print(soundex(name))  # Separate Soundex for "John" and "Sm
print(soundex("Altum"))

['A435']
['A435']


In [ ]:
print(doublemetaphone(name))
print(fuzzy.nysiis(name))

('ALTM', '')
ALTAN


In [ ]:
name="Altum"
print(doublemetaphone(name))
print(fuzzy.nysiis(name))

('ALTM', '')
ALTAN


In [ ]:
# Load the HTML file
model = SentenceTransformer('all-MiniLM-L6-v2')
html_file = "/content/TestExcel1.html"  # Replace with the name of your file

# Parse the HTML
with open(html_file, "r", encoding="utf-8") as file:
    soup = BeautifulSoup(file, "html.parser")

# Find all rows in the table
rows = soup.find_all("tr")

# Initialize a list to store the extracted data
data = []

# Extract the required information
for row in rows:
    columns = row.find_all("td")
    if len(columns) >= 3:  # Ensure at least 3 columns exist
        title_code = columns[0].get_text(strip=True)
        title_name = columns[1].get_text(strip=True)

        # Clean the title_name for Soundex processing
        title_name_s = title_name.replace(".", "")
        soundex_name = soundex(title_name_s)
        metaphone_name = doublemetaphone(title_name_s)[0]
        double_metaphone_name = doublemetaphone(title_name_s)
        nysiis_name = fuzzy.nysiis(title_name_s)
        vector = model.encode(metaphone_name).tolist()
        # Append the data to the list
        data.append([title_code, title_name, soundex_name, metaphone_name, double_metaphone_name[0], double_metaphone_name[1], nysiis_name,vector])
# Create a pandas DataFrame
print(data)
df = pd.DataFrame(data, columns=["Title_Code", "Title_Name", "Soundex_Name", "Metaphone_Name", "Double_Metaphone_Primary", "Double_Metaphone_Secondary", "NYSIIS_Name","vector"])

# Print the DataFrame
print(df)

# Convert the DataFrame to a dictionary (optional)
# dictionary = df.to_dict(orient="records")

# # Print the dictionary
# print(dictionary)

# # Save to a CSV file (optional)
# df.to_csv("titles_data.csv", index=False, encoding="utf-8")

# Save to a JSON file (optional)
df.to_json("titles_data.json", orient="records", force_ascii=False, indent=4)

NameError: name 'SentenceTransformer' is not defined

In [ ]:
df=pd.read_csv('titles_data.csv')
df

,Title Code,Title Name,Hindi Title,Soundex_Name,Metaphone_Name,Double_Metaphone_Primary,Double_Metaphone_Secondary,NYSIIS_Name
0,NOFHIN00556,JAN JAGRAN TIMES,NaN,"['J500', 'J265', 'T520']",JNJKRNTMS,JNJKRNTMS,ANJKRNTMS,JANJAGRANTAN
1,APENG03114,JAGRAN CITY PLUS,NaN,"['J265', 'C300', 'P420']",JKRNSTPLS,JKRNSTPLS,AKRNSTPLS,JAGRANCATAPL
2,BIHHIN05227,SAMPURNA JAGRAN,NaN,"['S516', 'J265']",SMPRNJKRN,SMPRNJKRN,NaN,SANPARNAJAGRAN
3,BIHHIN05120,DAINIK JAGRAN,NaN,"['D520', 'J265']",TNKJKRN,TNKJKRN,NaN,DANACJAGRAN
4,BIHHIN02261,VISHWA JAGRAN,NaN,"['V200', 'J265']",FXJKRN,FXJKRN,NaN,VASWAJAGRAN
...,...,...,...,...,...,...,...,...
219,UTTHIN05410,SATAT JAGRAN,सतत जागरण,"['S300', 'J265']",STTJKRN,STTJKRN,NaN,SATATJAGRAN
220,WBBEN08039,JAGRAN,जागरण,['J265'],JKRN,JKRN,AKRN,JAGRAN
221,WBHIN01556,DAINIK JAGRAN,NaN,"['D520', 'J265']",TNKJKRN,TNKJKRN,NaN,DANACJAGRAN
222,WBHIN00332,JAIN JAGRAN,NaN,"['J500', 'J265']",JNJKRN,JNJKRN,ANJKRN,JANJAGRAN


In [ ]:
from phonemizer import phonemize
from phonemizer.separator import Separator

# Define the input titles
titles = ["Morning Times", "Daily Herald", "India News"]

# Function to phonemize titles
def convert_to_phonemes(titles, language='en-us', backend='espeak'):
    phonetic_representations = []
    for title in titles:
        phonemes = phonemize(
            title,
            language=language,  # Set language (e.g., 'en-us' for English, 'hi' for Hindi, etc.)
            backend=backend,    # Backend to use ('espeak', 'festival', etc.)
            separator=Separator(phone=' ', syllable='|', word=' / ')  # Define separators
        )
        phonetic_representations.append(phonemes)
    return phonetic_representations

# Convert titles to phonemes
phonetic_titles = convert_to_phonemes(titles)

# Display the results
for title, phonemes in zip(titles, phonetic_titles):
    print(f"Original Title: {title}")
    print(f"Phonetic Representation: {phonemes}\n")


RuntimeError: espeak not installed on your system

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
import fuzzy
from metaphone import doublemetaphone
import json

# Load the embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')  # You can choose other models too

# Load the HTML file
html_file = "/content/TestExcel.html"  # Replace with the name of your file

# Parse the HTML
with open(html_file, "r", encoding="utf-8") as file:
    soup = BeautifulSoup(file, "html.parser")

# Find all rows in the table
rows = soup.find_all("tr")

# Initialize a list to store the extracted data
data = []

# Extract the required information
for row in rows:
    columns = row.find_all("td")
    if len(columns) >= 3:  # Ensure at least 3 columns exist
        title_code = columns[0].get_text(strip=True)
        title_name = columns[1].get_text(strip=True)

        # Clean the title_name for phonetic processing
        title_name_s = title_name.replace(".", "")
        soundex_name = soundex(title_name_s)
        metaphone_name = doublemetaphone(title_name_s)[0]
        double_metaphone_name = doublemetaphone(title_name_s)
        nysiis_name = fuzzy.nysiis(title_name_s)

        # Generate an embedding vector
        vector = model.encode(title_name_s).tolist()  # Convert to a list for JSON serialization

        # Append the data to the list
        data.append({
            "Title_Code": title_code,
            "Title_Name": title_name,
            "Soundex_Name": soundex_name,
            "Metaphone_Name": metaphone_name,
            "Double_Metaphone_Primary": double_metaphone_name[0],
            "Double_Metaphone_Secondary": double_metaphone_name[1],
            "NYSIIS_Name": nysiis_name,
            "vector": vector
        })

# Create the final JSON structure
output_json = {
    "rows": data
}

# Save to a JSON file
with open("titles_data_of_Final.json", "w", encoding="utf-8") as json_file:
    json.dump(output_json, json_file, ensure_ascii=False, indent=4)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Clean the title_name for phonetic processing
original_title = "DAINIK JAGRAN"
words = word_tokenize(original_title)
filtered_words = [word for word in words if word.lower() not in stopwords.words('english')]
sorted_sentence = sorted(filtered_words, key=str.lower)
title_after_sort = ' '.join(sorted_sentence)
print(original_title)
print(title_after_sort)
metaphone_name = doublemetaphone(title_after_sort)[0]
metaphone_name_without_sorting=doublemetaphone(original_title)[0]

# Generate an embedding vector
vector1 = model.encode(metaphone_name).tolist()
vector2 = model.encode(title_after_sort).tolist()
vector3=model.encode(metaphone_name_without_sorting).tolist()

# Append the data to the list
print({
    "Title_Name": original_title,
    "Title_Name_After_Sort": title_after_sort,
    "Metaphone_Name_After_Sort": metaphone_name,
    "Metaphone_Name_Without_Sorting": metaphone_name_without_sorting,
    "vector_of_metaphone": vector1,
    "vector_of_name": vector2,
    "vector_of_metaphone_without_sorting":vector3
})

DAINIK JAGRAN
DAINIK JAGRAN
{'Title_Name': 'DAINIK JAGRAN', 'Title_Name_After_Sort': 'DAINIK JAGRAN', 'Metaphone_Name_After_Sort': 'TNKJKRN', 'Metaphone_Name_Without_Sorting': 'TNKJKRN', 'vector_of_metaphone': [-0.1357230395078659, 0.04771851375699043, -0.009827629663050175, 0.006117030046880245, -0.04517020657658577, -0.05791391432285309, 0.05544978752732277, 0.04676637053489685, 0.0018298582872375846, -0.008285422809422016, 0.023374337702989578, -0.13259078562259674, 0.014894143678247929, 0.03252364322543144, -0.076437808573246, 0.055387288331985474, -0.01829991489648819, 0.06453181058168411, -0.05470190942287445, -0.023506980389356613, -0.20546959340572357, -0.012894083745777607, 0.08223798871040344, -0.03666933253407478, 0.03954857215285301, 0.012014120817184448, 0.028474073857069016, 0.0541166253387928, 0.028007956221699715, -0.017109880223870277, -0.02411518432199955, 0.03967680037021637, 0.0295877605676651, 0.0007225397857837379, 0.004951282870024443, -0.0012186737731099129, -0.

In [ ]:
trans_word_array=[]
translator = Translator()
input="SACH KI AWAZ"
words=input.split()
print(words)

filtered_words = [word for word in words if word.lower() not in stopwords.words('english')]
print(filtered_words)
for i in filtered_words:
  print(i)
  out = e.translit_word(i, topk=5)
  print(out)
  translated_string1 = translator.translate(out['hi'][0], dest='en').text
  print(translated_string1)
  trans_word_array.append(translated_string1)
# Here please merge the hindi_wrod_array to one single sentence
print(trans_word_array)
# trans_sentence = " ".join(trans_word_array)
# print(trans_sentence)
trans_sorted_sentence = sorted(trans_word_array, key=str.lower)
title_after_sort = ' '.join(trans_sorted_sentence)
print(title_after_sort)

['SACH', 'KI', 'AWAZ']
['SACH', 'KI', 'AWAZ']
SACH
{'hi': ['सच', 'साच', 'सैच', 'साक', 'सैक']}
Truth
KI
{'hi': ['की', 'कि', 'काई', 'कीई', 'काइ']}
of
AWAZ
{'hi': ['आवाज', 'आवाज़', 'अवाज़', 'अवाज', 'अवज़']}
Sound
['Truth', 'of', 'Sound']
of Sound Truth


In [ ]:
trans_word_array=[]
translator = Translator()
input="JAMHURIYAT KI AWAZ"
words = re.split(r'[ \-]', input)
print(words)

for i in words:
  out = e.translit_word(i, topk=5)
  print(out['hi'])
  translated_string1 = translator.translate(out['hi'][0], dest='en').text
  print(translated_string1)
  trans_word_array.append(translated_string1)
print(trans_word_array)
filtered_words = [word for word in trans_word_array if word.lower() not in stopwords.words('english')]
trans_sorted_sentence = sorted(filtered_words, key=str.lower)
title_after_sort = ' '.join(trans_sorted_sentence)
print(title_after_sort)

['JAMHURIYAT', 'KI', 'AWAZ']
['जम्हूरियत', 'जम्हुरियत', 'जमहूरियत', 'जमहुरियत', 'जम्हुरीयत']
democracy
['की', 'कि', 'काई', 'कीई', 'काइ']
of
['आवाज', 'आवाज़', 'अवाज़', 'अवाज', 'अवज़']
Sound
['democracy', 'of', 'Sound']
democracy Sound


In [ ]:
# Load the embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Directory containing the HTML files
html_files = [f"/content/TestExcel{i}.html" for i in range(0, 2)]  # List of file names

# Initialize a list to store the extracted data from all files
all_data = []

# Process each HTML file
for html_file in html_files:
    print(html_file)
    # Parse the HTML
    with open(html_file, "r", encoding="utf-8") as file:
        soup = BeautifulSoup(file, "html.parser")

    # Find all rows in the table
    rows = soup.find_all("tr")

    # Extract the required information
    for row in rows:
        columns = row.find_all("td")
        title_code = columns[0].get_text(strip=True)
        title_name = columns[1].get_text(strip=True)

        # Clean the title_name for phonetic processing
        original_title = title_name.replace(".", "")
        words = word_tokenize(original_title)
        filtered_words = [word for word in words if word.lower() not in stopwords.words('english')]
        sorted_sentence = sorted(filtered_words, key=str.lower)
        title_after_sort = ' '.join(sorted_sentence)
        # print(original_title)
        # print(title_after_sort)
        metaphone_name = doublemetaphone(title_after_sort)[0]
        metaphone_name_without_sorting=doublemetaphone(original_title)[0]


        vector1 = model.encode(metaphone_name).tolist()
        vector2 = model.encode(title_after_sort).tolist()
        vector3 = model.encode(metaphone_name_without_sorting).tolist()

        # Append the data to the list
        all_data.append({
          "Title_Code": title_code,
          "Title_Name": original_title,
          "Title_Name_After_Sort": title_after_sort,
          "Metaphone_Name_After_Sort": metaphone_name,
          "Metaphone_Name_Without_Sorting": metaphone_name_without_sorting,
          "vector_of_metaphone": vector1,
          "vector_of_name": vector2,
          "vector_of_metaphone_without_sorting":vector3
          })

            # words = re.split(r'[ \-]', original_title)
            # translated_string1=""
            # trans_word_array=[]
            # for i in words:
            #   out = e.translit_word(i, topk=5)
            #   if (out and len(out['hi'])>0):
            #     try:
            #       translated_string1 = translator.translate(out['hi'][0], dest='en').text
            #       trans_word_array.append(translated_string1)
            #     except:
            #       print(original_title)
            #       # print("ERRORRRR")
            #       # print(out['hi'])
            #   else:
            #     print(out)
            #     print(len(out['hi']))
            #     translated_string1 = translator.translate(i, dest='en').text
            #     trans_word_array.append(translated_string1)
            # filtered_words = [word for word in trans_word_array if word.lower() not in stopwords.words('english')]
            # trans_sorted_sentence = sorted(filtered_words, key=str.lower)
            # trans_title_after_sort = ' '.join(trans_sorted_sentence)
            # print(trans_title_after_sort)
            # print(title_after_sort)
            # filtered_words = [word for word in words if word.lower() not in stopwords.words('english')]
            # for i in filtered_words:
            #   out = e.translit_word(i, topk=5)
            #   try:
            #     translated_string1 = translator.translate(out['hi'][0], dest='en').text
            #     trans_word_array.append(translated_string1)
            #   except:
            #     print(original_title)
            #     print("ERRORRRR")
            #     print(out['hi'])
            # trans_sorted_sentence = sorted(trans_word_array, key=str.lower)
            # trans_title_after_sort = ' '.join(trans_sorted_sentence)
            # print(f"original_Title {original_title}")
            # print(trans_title_after_sort)

            # Generate an embedding vector


# Create the final JSON structure
output_json = {
    "rows": all_data
}

# Save to a single JSON file
output_file = "three_vectors_0to2.json"
with open(output_file, "w", encoding="utf-8") as json_file:
    json.dump(output_json, json_file, ensure_ascii=False, indent=4)

print(f"Combined data has been saved to {output_file}")


/content/TestExcel0.html
/content/TestExcel1.html
Combined data has been saved to three_vectors_0to2.json


In [ ]:
all_data = []
model = SentenceTransformer('all-MiniLM-L6-v2')
title_code = "1234556"
title_name = "GRAMIN PRATIDIN"

# Clean the title_name for phonetic processing
title_name_s = title_name.replace(".", "")
soundex_name = soundex(title_name_s)
metaphone_name = doublemetaphone(title_name_s)[0]
double_metaphone_name = doublemetaphone(title_name_s)
nysiis_name = fuzzy.nysiis(title_name_s)
vector1 = model.encode(metaphone_name).tolist()  # Convert to a list for JSON serialization
vector2 = model.encode(title_name_s).tolist()
all_data.append({
  "Title_Code": title_code,
  "Title_Name": title_name,
  "Soundex_Name": soundex_name,
  "Metaphone_Name": metaphone_name,
  "Double_Metaphone_Primary": double_metaphone_name[0],
  "Double_Metaphone_Secondary": double_metaphone_name[1],
  "NYSIIS_Name": nysiis_name,
  "vector_of_metaphone": vector1,
  "vector_of_name": vector2
})
output_json = {
    "rows": all_data
}

output_file = "Single_vector.json"
with open(output_file, "w", encoding="utf-8") as json_file:
    json.dump(output_json, json_file, ensure_ascii=False, indent=4)

print(f"Combined data has been saved to {output_file}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Combined data has been saved to Single_vector.json


In [ ]:
!pip install Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 35.6 MB/s eta 0:00:00


In [ ]:
import Levenshtein

In [ ]:
name1 = "VIDHYA"
name2 = "SHOSIT"

# Get primary metaphone encodings
meta1 = doublemetaphone(name1)[0]
meta2 = doublemetaphone(name2)[0]

print(meta1, meta2)
distance = Levenshtein.distance(meta1, meta2)
print(f"Levenshtein Distance: {distance}")

FT XST
Levenshtein Distance: 2


In [ ]:
from metaphone import doublemetaphone
import Levenshtein

# Given titles
query_title = "VISHWA JAGRAN"
titles = ["SHOSIT JAGRAN", "VIDISHA JAGRAN"]

# Metaphone encodings
query_meta = doublemetaphone(query_title)[0]
metaphones = [doublemetaphone(title)[0] for title in titles]

# Compute Levenshtein distances
distances = [Levenshtein.distance(query_meta, meta) for meta in metaphones]

# Print distances
for title, meta, dist in zip(titles, metaphones, distances):
    print(f"Title: {title}, Metaphone: {meta}, Levenshtein Distance: {dist}")


Title: SHOSIT JAGRAN, Metaphone: XSTJKRN, Levenshtein Distance: 3
Title: VIDISHA JAGRAN, Metaphone: FTXJKRN, Levenshtein Distance: 1


In [ ]:
import numpy as np

# Example scores from Milvus and Levenshtein distances
vector_scores = [0.78, 0.77]  # Milvus similarity scores
levenshtein_distances = [3, 1]  # Levenshtein distances

# Normalize Levenshtein distances (lower is better)
max_distance = max(levenshtein_distances)
normalized_levenshtein = [1 - (dist / max_distance) for dist in levenshtein_distances]

# Combine scores using weights
w1, w2 = 0.5, 0.5  # Adjust weights as per your preference
final_scores = [
    w1 * vector + w2 * leven for vector, leven in zip(vector_scores, normalized_levenshtein)
]

# Print results
titles = ["SHOSIT JAGRAN", "VIDISHA JAGRAN"]
for title, score in zip(titles, final_scores):
    print(f"Title: {title}, Combined Score: {score}")


Title: SHOSIT JAGRAN, Combined Score: 0.39
Title: VIDISHA JAGRAN, Combined Score: 0.7183333333333334
